This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2019-01-20&end_date=2019-01-20&api_key=' + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.content

'{"dataset":{"id":10095370,"dataset_code":"AFX_X","database_code":"FSE","name":"Carl Zeiss Meditec (AFX_X)","description":"Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.\\u003cbr\\u003e\\u003cbr\\u003eTrading System: Xetra\\u003cbr\\u003e\\u003cbr\\u003eISIN: DE0005313704","refreshed_at":"2019-01-22T22:30:10.278Z","newest_available_date":"2019-01-22","oldest_available_date":"2000-06-07","column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"frequency":"daily","type":"Time Series","premium":false,"limit":null,"transform":null,"column_index":null,"start_date":"2019-01-20","end_date":"2019-01-20","data":[],"collapse":null,"order":null,"database_id":6129}}'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# request data for AFX_X on Frankfurt Stock Exchange for 2017
afx_x_req = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)

# convert response to dictionary
afx_x = afx_x_req.json()

# store headers and data
afx_header = afx_x['dataset']['column_names']
afx_x = afx_x['dataset']['data']

# view index for each header to map to data
for index, header in enumerate(afx_header):
    print str(index) + ': ' + header

0: Date
1: Open
2: High
3: Low
4: Close
5: Change
6: Traded Volume
7: Turnover
8: Last Price of the Day
9: Daily Traded Units
10: Daily Turnover


In [6]:
# create and populate a list of opening prices
opens = []
for day in afx_x:
    if day[1] != None:
        opens.append(day[1])
        
# print highest and lowest opening prices
print 'Highest opening: ' + str(max(opens))
print 'Lowest opening: ' + str(min(opens))

Highest opening: 53.11
Lowest opening: 34.0


In [7]:
# create and populate a list of changes
changes = []
for day in afx_x:
    changes.append(day[2]-day[3])
    
# print highest daily change
print 'Highest change (same day high - low): ' + str(max(changes))

Highest change (same day high - low): 2.81


In [8]:
# create and populate a list of closing prices
closes = []
for day in afx_x:
    closes.append(day[4])
closes.reverse()

# clone list of closing prices
closes_next_day = list(closes)

# delete last value to closing prices
del closes[-1]

# delete first value of cloned closing prices
del closes_next_day[0]

# element-wise difference between closing price for one day and the next day
daily_change = zip(closes, closes_next_day)
two_day_change = [abs(day_two - day_one) for day_one, day_two in daily_change]

# print highest magnitude difference
print 'Highest change between consecutive trading day closing prices: ' + str(max(two_day_change))

Highest change between consecutive trading day closing prices: 2.56


In [9]:
# create and populate a list of trading volumes
volumes = []
for day in afx_x:
    volumes.append(day[6])
    
# calculate and print mean of trading volumes
print 'Mean daily trading volume: ' + str(round(sum(volumes)/len(volumes),2))

Mean daily trading volume: 89124.34


In [10]:
def median(list):
    """This function accepts a list of numbers and returns the median"""
    list.sort()
    l = len(list)
    if l % 2 == 0:
        med = float((list[l / 2] + list[l / 2 - 1])) / 2
    else:
        med = list[int(l / 2)]
    return med

# calculate and print median of trading volumes
print 'Median daily trading volume: ' + str(median(volumes))

Median daily trading volume: 76286.0
